In [5]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
data_dictionary = pd.read_csv('data_dictionary.csv')
productivity_data = pd.ExcelFile('Manufacturing_Line_Productivity.xlsx')


In [39]:
sheet_names = productivity_data.sheet_names
sheet_names

['Line productivity', 'Products', 'Downtime factors', 'Line downtime']

In [119]:
line_productivity_df = productivity_data.parse('Line productivity')
line_productivity_df.head()
products_df = productivity_data.parse('Products')
products_df.head()

line_productivity_df = line_productivity_df[:-1]

In [41]:
# Ensure Date is in datetime format
line_productivity_df['Date'] = pd.to_datetime(line_productivity_df['Date'])

# Combine Date with Start Time and End Time to create full datetime objects
line_productivity_df['Start DateTime'] = line_productivity_df['Date'] + pd.to_timedelta(line_productivity_df['Start Time'].astype(str))
line_productivity_df['End DateTime'] = line_productivity_df['Date'] + pd.to_timedelta(line_productivity_df['End Time'].astype(str))

# Calculate Batch Time as the difference between Start DateTime and End DateTime
line_productivity_df['Batch Time'] = (line_productivity_df['End DateTime'] - line_productivity_df['Start DateTime']).dt.total_seconds() / 60  # Duration in minutes
line_productivity_df.head()

,Date,Product,Batch,Operator,Start Time,End Time,Start DateTime,End DateTime,Batch Time
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00,2024-08-29 11:50:00,2024-08-29 14:05:00,135.0
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00,2024-08-29 14:05:00,2024-08-29 15:45:00,100.0
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00,2024-08-29 15:45:00,2024-08-29 17:35:00,110.0
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00,2024-08-29 17:35:00,2024-08-29 19:15:00,100.0
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00,2024-08-29 19:15:00,2024-08-29 20:39:00,84.0


In [42]:
line_productivity_df = line_productivity_df.merge(products_df[['Product','Min batch time']], on='Product', how='left')



,Date,Product,Batch,Operator,Start Time,End Time,Start DateTime,End DateTime,Batch Time,Min batch time
0,2024-08-29,OR-600,422111,Mac,11:50:00,14:05:00,2024-08-29 11:50:00,2024-08-29 14:05:00,135.0,60
1,2024-08-29,LE-600,422112,Mac,14:05:00,15:45:00,2024-08-29 14:05:00,2024-08-29 15:45:00,100.0,60
2,2024-08-29,LE-600,422113,Mac,15:45:00,17:35:00,2024-08-29 15:45:00,2024-08-29 17:35:00,110.0,60
3,2024-08-29,LE-600,422114,Mac,17:35:00,19:15:00,2024-08-29 17:35:00,2024-08-29 19:15:00,100.0,60
4,2024-08-29,LE-600,422115,Charlie,19:15:00,20:39:00,2024-08-29 19:15:00,2024-08-29 20:39:00,84.0,60


In [46]:
line_productivity_df.tail()

,Date,Product,Batch,Operator,Start Time,End Time,Start DateTime,End DateTime,Batch Time,Min batch time
32,2024-09-02,RB-600,422143,Dennis,10:20:00,12:18:00,2024-09-02 10:20:00,2024-09-02 12:18:00,118.0,60
33,2024-09-02,CO-2L,422144,Dennis,12:18:00,14:50:00,2024-09-02 12:18:00,2024-09-02 14:50:00,152.0,98
34,2024-09-02,CO-2L,422145,Charlie,14:50:00,16:50:00,2024-09-02 14:50:00,2024-09-02 16:50:00,120.0,98
35,2024-09-02,CO-2L,422146,Charlie,16:50:00,19:30:00,2024-09-02 16:50:00,2024-09-02 19:30:00,160.0,98
36,2024-09-02,CO-2L,422147,Charlie,19:30:00,22:55:00,2024-09-02 19:30:00,2024-09-02 22:55:00,205.0,98


# 1-  Current Line Efficiency 

In [99]:
total_actual_time = line_productivity_df['Batch Time'].sum()
total_min_time = line_productivity_df['Min batch time'].sum()
total_efficieny = total_min_time/total_actual_time

total_efficieny, total_min_time, total_actual_time

(0.6362660944206009, 2372, 3728.0)

# 2- Are any operators underperforming?

In [50]:
# Group data by operator and calculate total actual and minimum times
operator_stats = line_productivity_df.groupby('Operator').agg(
    total_actual_time=('Batch Time', 'sum'),
    total_min_time=('Min batch time', 'sum')
)

# Calculate efficiency for each operator
operator_stats['Efficiency'] = operator_stats['total_min_time'] / operator_stats['total_actual_time']

# Calculate the average efficiency
average_efficiency = operator_stats['Efficiency'].mean()

# Identify underperforming operators (efficiency below average)
underperforming_operators = operator_stats[operator_stats['Efficiency'] < average_efficiency]

# Display the underperforming operators
underperforming_operators


,total_actual_time,total_min_time,Efficiency
Operator,,,
Mac,720.0,420,0.583333


# 3- What are the leading factors for downtime?


In [109]:
downtime_factors_df = productivity_data.parse('Downtime factors')
downtime_factors_df.head()
line_downtime_df = productivity_data.parse('Line downtime')
line_downtime_df.columns = line_downtime_df.iloc[0]
line_downtime_df = line_downtime_df[1:]
line_downtime_df.head()

,Batch,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
1,422111,NaN,60.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
2,422112,NaN,20.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN
3,422113,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,422114,NaN,NaN,NaN,25.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN
5,422115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN


In [110]:
# Create a mapping dictionary: {factor_code: description}
factor_mapping = dict(zip(downtime_factors_df['Factor'], downtime_factors_df['Description']))

# Replace 'Downtime Factor' codes in the line downtime sheet with their descriptions
line_downtime_df.rename(columns=factor_mapping, inplace=True)

line_downtime_df.head()

,Batch,Emergency stop,Batch change,Labeling error,Inventory shortage,Product spill,Machine adjustment,Machine failure,Batch coding error,Conveyor belt jam,Calibration error,Label switch,Other
1,422111,NaN,60.0,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN
2,422112,NaN,20.0,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN
3,422113,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,422114,NaN,NaN,NaN,25.0,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN
5,422115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN


In [111]:
line_downtime_df.sum().sort_values(ascending=False)

0
Batch                 16040921
Machine adjustment       332.0
Machine failure          254.0
Inventory shortage       225.0
Batch change             160.0
Batch coding error       145.0
Other                     74.0
Product spill             57.0
Calibration error         49.0
Labeling error            42.0
Label switch              33.0
Conveyor belt jam         17.0
Emergency stop             0.0
dtype: object

# 4 - Do any operators struggle with particular types of operator error?

In [113]:
# Step 1: Filter downtime factors for operator errors
operator_error_factors = downtime_factors_df[downtime_factors_df['Operator Error'] == 'Yes']['Description'].tolist()
operator_error_factors

['Batch change',
 'Product spill',
 'Machine adjustment',
 'Batch coding error',
 'Calibration error',
 'Label switch']

In [114]:
line_downtime_long = line_downtime_df.melt(id_vars=['Batch'], var_name='Downtime Factor', value_name='Downtime (mins)')

line_downtime_filtered = line_downtime_long[line_downtime_long['Downtime Factor'].isin(operator_error_factors)]
line_downtime_filtered.head()

,Batch,Downtime Factor,Downtime (mins)
38,422111,Batch change,60.0
39,422112,Batch change,20.0
40,422113,Batch change,50.0
41,422114,Batch change,NaN
42,422115,Batch change,NaN


In [120]:

merged_operator_error_data = line_downtime_filtered.merge(
    line_productivity_df[['Batch', 'Operator']], on='Batch', how='left'
)
merged_operator_error_data.head()

,Batch,Downtime Factor,Downtime (mins),Operator
0,422111,Batch change,60.0,Mac
1,422112,Batch change,20.0,Mac
2,422113,Batch change,50.0,Mac
3,422114,Batch change,NaN,Mac
4,422115,Batch change,NaN,Charlie


In [126]:
# Group by Operator and Downtime Factor, and calculate total downtime minutes
operator_downtime_summary = merged_operator_error_data.groupby(['Operator', 'Downtime Factor'])['Downtime (mins)'].sum().reset_index()

operator_downtime_summary = operator_downtime_summary.sort_values(['Downtime (mins)'], ascending=[False])

operator_downtime_summary


,Operator,Downtime Factor,Downtime (mins)
18,Mac,Batch change,130.0
16,Dennis,Machine adjustment,120.0
4,Charlie,Machine adjustment,118.0
10,Dee,Machine adjustment,79.0
1,Charlie,Batch coding error,44.0
19,Mac,Batch coding error,40.0
7,Dee,Batch coding error,30.0
8,Dee,Calibration error,25.0
13,Dennis,Batch coding error,24.0
2,Charlie,Calibration error,24.0
